In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow import keras
from scipy import sparse
import pandas as pd
import os
from google.colab import drive

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
os.chdir("/content/drive/MyDrive/lendingClubLoanData/notebooks")

In [4]:
! ls

Exploration.ipynb  Preprocessing.ipynb	Train_Evaluate.ipynb


In [5]:
Ytrain = pd.read_csv("../../dataset/preprocessed_data/Ytrain.csv")
Ytest = pd.read_csv("../../dataset/preprocessed_data/Ytest.csv")
Xtrain_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xtrain_preprocessed.npz")
Xtest_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xtest_preprocessed.npz")

In [6]:
# Creation du modele

model = Sequential()
model.add(Input((Xtrain_preprocessed.shape[1],)))
model.add(Dense(units = 200 , activation = "relu"))
model.add(Dense(units = 60))
model.add(BatchNormalization())
model.add(Dense(units = 30, activation = "relu"))
model.add(Dense(units = 1, activation = "sigmoid"))


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 200)            │    10,051,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │        12,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 60)             │           240 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 30)             │         1,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,065,561 (38.40 MB)

 Trainable params: 10,065,441 (38.40 MB)

 Non-trainable params: 120 (480.00 B)

In [8]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [9]:
Ytrain.shape

(33786, 1)

In [10]:
history = model.fit(
    Xtrain_preprocessed, Ytrain,
    validation_data = (Xtest_preprocessed, Ytest),
    epochs = 2,
    batch_size = 32,
    verbose = True
)

Epoch 1/2
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 32s 27ms/step - accuracy: 0.9798 - loss: 0.0643 - val_accuracy: 0.9959 - val_loss: 0.0157
Epoch 2/2
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9971 - loss: 0.0105 - val_accuracy: 0.9932 - val_loss: 0.0193


# Evaluation du modele

In [11]:
Xval_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xval_preprocessed.npz")
Yval = pd.read_csv("../../dataset/preprocessed_data/Yval.csv")

predictions = model.predict(Xval_preprocessed)

493/493 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step


In [12]:
Yval

,loan_status
0,1
1,1
2,1
3,1
4,1
...,...
15762,1
15763,1
15764,1
15765,1


In [13]:
print(classification_report(Yval, predictions.round()))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      2467
           1       1.00      1.00      1.00     13300

    accuracy                           1.00     15767
   macro avg       0.99      0.99      0.99     15767
weighted avg       1.00      1.00      1.00     15767



In [14]:
print(confusion_matrix(Yval, predictions.round()))

[[ 2440    27]
 [   42 13258]]


In [15]:
Yval.value_counts(normalize = True)

,proportion
loan_status,
1,0.843534
0,0.156466


In [16]:
model.save("../models/model1.keras")